# RSA

In [4]:
import random


def get_primes(start, stop):
    print("start:", start, ", stop:", stop)
    if start >= stop:
        return []

    primes = [2]

    for n in range(3, stop + 1, 2):
        for p in primes:
            if n % p == 0:
                break
        else:
            primes.append(n)

    while primes and primes[0] < start:
        del primes[0]

    return primes


def are_relatively_prime(a, b):
    for n in range(2, min(a, b) + 1):
        if a % n == b % n == 0:
            return False

    return True


def make_key_pair(length):
    if length < 4:
        raise ValueError('cannot generate a key of length less '
                         'than 4 (got {!r})'.format(length))

    n_min = 1 << (length - 1)
    n_max = (1 << length) - 1
    print("n_min", n_min, ", n_max:", n_max)

    start = 1 << (length // 2 - 1)
    stop = 1 << (length // 2 + 1)
    primes = get_primes(start, stop)

    while primes:
        print("Primes:", primes)
        p = random.choice(primes)
        primes.remove(p)
        q_candidates = [q for q in primes
                        if n_min <= p * q <= n_max]
        if q_candidates:
            q = random.choice(q_candidates)
            break
    else:
        raise AssertionError("cannot find 'p' and 'q' for a key of "
                             "length={!r}".format(length))

    stop = (p - 1) * (q - 1)
    for e in range(3, stop, 2):
        if are_relatively_prime(e, stop):
            break
    else:
        raise AssertionError("cannot find 'e' with p={!r} "
                             "and q={!r}".format(p, q))

    for d in range(3, stop, 2):
        if d * e % stop == 1:
            break
    else:
        raise AssertionError("cannot find 'd' with p={!r}, q={!r} "
                             "and e={!r}".format(p, q, e))

    print("p:", p, ", q:", q)
    # That's all. We can build and return the public and private keys.
    return p * q, e, d


def encrypt(data, e, n):

    if isinstance(data, str):
        data = data.encode('ascii')

    encrypted_data = []
    for byte in data:
        encrypted_data.append(pow(byte, e, n))

    return encrypted_data


def decrypt(data, d, n):
    decrypted_data = []

    for byte in data:
        decrypted_data.append(pow(byte, d, n))

    return bytes(decrypted_data).decode("utf8")

In [6]:
def write_to_file(file_name, data):
    file = open(file_name, "w")
    file.write(str(data))
    file.close()


file = open("initial.txt", "r")
data = file.read()
file.close()
print("Data:", data)

n, e, d = make_key_pair(10)
print("n:", n, ", e:", e, ", d", d)

encrypted_data = encrypt(data, e, n)
print("\nEncrypted data:\n{}".format(encrypted_data))
write_to_file("encrypted.txt", encrypted_data)

decrypted_data = decrypt(encrypted_data, d, n)
print("\nDecrypted data:\n{}".format(decrypted_data))
write_to_file("decrypted.txt", decrypted_data)
   

Data: What's up Rumyn?) Have a nice day!

n_min 512 , n_max: 1023
start: 16 , stop: 64
Primes: [17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61]
p: 23 , q: 41
n: 943 , e: 3 , d 587

Encrypted data:
[289, 808, 792, 231, 853, 759, 706, 399, 801, 706, 656, 399, 290, 607, 427, 152, 82, 706, 763, 792, 326, 545, 706, 792, 706, 427, 564, 895, 545, 706, 420, 792, 607, 103, 57]

Decrypted data:
What's up Rumyn?) Have a nice day!

